In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# NPBのプロセス数
npb_process :list[int] = [2, 4, 8, 16, 32, 64, 128, 256]
train_npb_process :list[int] = npb_process[:-1]
test_npb_process :list[int] = npb_process[-1:]
# NPBのCGの初期変数
cg_na: list[int] = [14000, 30000, 75000, 100000, 1500000]
cg_nonzer: list[int] = [11, 12, 13, 14, 15, 18, 21]
cg_niter: list[int] = [15, 30, 75, 90, 100]
cg_shift: list[int] = [20, 40, 60, 80, 110, 200]

train_cg_na: list[int] = cg_na[:-1]
train_cg_nonzer: list[int] = cg_nonzer[:-1]
train_cg_niter: list[int] = cg_niter[:-1]
train_cg_shift: list[int] = cg_shift[:-1]

test_cg_na: list[int] = cg_na[-1:]
test_cg_nonzer: list[int] = cg_nonzer[-1:]
test_cg_niter: list[int] = cg_niter[-1:]
test_cg_shift: list[int] = cg_shift[-1:]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

# Extra-Pのオプション
modelerNames: list[str] = [
    # "refining", 
    "multi-parameter",
    "default", 
    # "basic --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1"
    ]

modelerOption: str = """ --options \#spm=Basic \#spo=poly_exponents=-1,0,1,2,3,log_exponents=0,1,force_combination_exponents=1,allow_negative_exponents=True"""

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]
list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

/workspace


DEBUG:__main__:hello
DEBUG:lib.lab_lib:hello


# TODO

* `Exclusive` の予測
    * Extra-P 単体でのモデル構築
    * Extra-P と Extra-P の組み合わせによるモデル構築
    * Extra-P 単体での予測
    * Extra-P と Extra-p の組み合わせによる予測
* `Inclusive` の予測
    * Extra-P 単体でのモデル構築
    * Extra-P と Extra-P の組み合わせによるモデル構築
    * Extra-P 単体での予測
    * Extra-P と Extra-p の組み合わせによる予測
* `#Call` の予測
    * Extra-P 単体でのモデル構築
    * Extra-P と Extra-P の組み合わせによるモデル構築
    * Extra-P 単体での予測
* `#Subrs` の予測
    * Extra-P 単体でのモデル構築
    * Extra-P と Extra-P の組み合わせによるモデル構築
    * Extra-P 単体での予測
    * Extra-P と Extra-p の組み合わせによる予測



In [2]:
# 関数コール回数予測のためのモデル

resVar = "#Call"
expVar: list[str] = ["process", "iteration", "size"]


dict_symbols_lulesh = {}
for elem in expVar:
    dict_symbols_lulesh[elem] = symbols(elem, real=True)

target_env_lulesh = [
    (dict_symbols_lulesh["size"], test_lulesh_sizes[-1]),
    (dict_symbols_lulesh["iteration"], test_lulesh_iterations[-1]),
    (dict_symbols_lulesh["process"], test_lulesh_processes[-1]),
]

modelerName: str = "multi-parameter"
modelerOption: str = """ --options \#spm=Basic \#spo=poly_exponents=-1,0,1,2,3,log_exponents=0,1,force_combination_exponents=1,allow_negative_exponents=True"""

print(f"resVar = {resVar}")

trainDF_lulesh: pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=train_lulesh_processes,
    list_iteration=train_lulesh_iterations,
    list_size=train_lulesh_sizes,
    list_csvDir=list_csvDir,
    resVar=resVar,
)

functionNames_lulesh: list[str] = sorted(list(set(trainDF_lulesh["Name"])))

dict_functionName_model_call: dict[str, str] = {}

for functionName in functionNames_lulesh:
    trainDF_perFunc: pd.DataFrame = trainDF_lulesh[
        trainDF_lulesh["Name"] == functionName
    ].reset_index()

    model_fromExtraP_all = get_ExtraP_model(
        inputDF_perFunc=trainDF_perFunc,
        expVar=expVar,
        resVar=resVar,
        functionName=functionName,
        benchmarkName="lulesh",
        modelerName=modelerName,
        modelerOption=modelerOption,
        dict_symbols=dict_symbols_lulesh,
    )
    dict_functionName_model_call[functionName] = model_fromExtraP_all

resVar = #Call


In [3]:
list_resVar: list[str] = ["Exclusive", "Inclusive", "#Subrs"]
dict_resVar_functionName_models_all :dict = {}
dict_resVar_functionName_models_perCall :dict = {}

for resVar in list_resVar:
    pass

    print(f"resVar = {resVar}")

    trainDF_lulesh: pd.DataFrame = ret_averaged_rawDF_lulesh(
        list_process=train_lulesh_processes,
        list_iteration=train_lulesh_iterations,
        list_size=train_lulesh_sizes,
        list_csvDir=list_csvDir,
        resVar=resVar,
    )

    functionNames_lulesh: list[str] = sorted(list(set(trainDF_lulesh["Name"])))

    dict_functionName_model_all: dict[str, str] = {}
    dict_functionName_model_perCall: dict[str, str] = {}

    for functionName in functionNames_lulesh:
        trainDF_perFunc: pd.DataFrame = trainDF_lulesh[
            trainDF_lulesh["Name"] == functionName
        ].reset_index()

        model_fromExtraP_all = get_ExtraP_model(
            inputDF_perFunc=trainDF_perFunc,
            expVar=expVar,
            resVar=resVar,
            functionName=functionName,
            benchmarkName="lulesh",
            modelerName=modelerName,
            modelerOption=modelerOption,
            dict_symbols=dict_symbols_lulesh,
        )
        dict_functionName_model_all[functionName] = model_fromExtraP_all

        model_fromExtraP_perCall = get_ExtraP_model(
            inputDF_perFunc=trainDF_perFunc,
            expVar=expVar,
            resVar=f"{resVar}PerCall",
            functionName=functionName,
            benchmarkName="lulesh",
            modelerName=modelerName,
            modelerOption=modelerOption,
            dict_symbols=dict_symbols_lulesh,
        )
        dict_functionName_model_perCall[functionName] = model_fromExtraP_perCall

    dict_resVar_functionName_models_all[resVar] = dict_functionName_model_all
    dict_resVar_functionName_models_perCall[resVar] = dict_functionName_model_perCall

resVar = Exclusive
resVar = Inclusive
resVar = #Subrs


In [8]:
dict_resultDF: dict[str, pd.DataFrame] = {}
for resVar in list_resVar:

    testDF_lulesh = ret_averaged_rawDF_lulesh(
        list_process=test_lulesh_processes,
        list_iteration=test_lulesh_iterations,
        list_size=test_lulesh_sizes,
        list_csvDir=list_csvDir,
        resVar=resVar,
    )

    functionNames_lulesh: list[str] = sorted(list(set(testDF_lulesh["Name"])))
    _list_series: list[pd.Series] = []

    for functionName in functionNames_lulesh:
        tmp_series: pd.Series = pd.Series(dtype="object")

        testDF_perFunc: pd.DataFrame = testDF_lulesh[
            (testDF_lulesh["Name"] == functionName)
            & (testDF_lulesh["process"] == test_lulesh_processes[-1])
            & (testDF_lulesh["iteration"] == test_lulesh_iterations[-1])
            & (testDF_lulesh["size"] == test_lulesh_sizes[-1])
        ].reset_index()

        if len(testDF_perFunc) == 1:
            tmp_series[f"real_{resVar}"] = testDF_perFunc.loc[0][resVar]
            tmp_series["functionName"] = functionName
            tmp_series[f"predicted_{resVar}"] = (
                dict_resVar_functionName_models_all[resVar][functionName]
                .subs(target_env_lulesh)
                .evalf()
            )

            # コール回数予測
            _predicted_call: float
            try:
                _predicted_call = (
                    dict_functionName_model_call[functionName]
                    .subs(target_env_lulesh)
                    .evalf()
                )
            except:
                _predicted_call = dict_functionName_model_call[functionName]
            # 予測対象値の１コール当たりの値
            _target_num_perCall: float = (
                dict_resVar_functionName_models_perCall[resVar][functionName]
                .subs(target_env_lulesh)
                .evalf()
            )
            # 予測されたコール回数 * 予測対象値の1コールあたりの値
            _predicted_num: float = _predicted_call * _target_num_perCall

            # print(f"dict_functionName_model_call[functionName]={dict_functionName_model_call[functionName]}, _target_num_perCall={_target_num_perCall}, _predicted_call={_predicted_call}")

            tmp_series[f"predicted_{resVar}_from_call"] = _predicted_num
            tmp_series["predicted_call"] = _predicted_call
            tmp_series["real_call"] = testDF_perFunc.loc[0][resVar]
            _list_series.append(tmp_series)

    print(resVar)
    dict_resultDF[resVar] = pd.DataFrame(data=_list_series)

Exclusive
Inclusive
#Subrs


In [5]:
list_resVar: list[str] = ["Exclusive", "Inclusive", "#Subrs"]

print(dict_resultDF.keys())

dict_resultDF[list_resVar[0]]

dict_keys(['Exclusive', 'Inclusive', '#Subrs'])


,real_Exclusive,functionName,predicted_Exclusive,predicted_Exclusive_from_call,predicted_call,real_call
0,3.366667e-06,.TAU_application,3.58708333333333e-6,2.10570898355123e-9,0.000999999999999997,3.366667e-06
1,5.133010e+02,MPI_Allreduce(),437.915990349888,12.1247559208733,69.1144138888889,5.133010e+02
2,3.833333e-02,MPI_Barrier(),0.000816330555555555,0.0185004563418933,0.662485549774719,3.833333e-02
3,9.000000e-03,MPI_Comm_rank(),0.0165749444775296,5.75717034518277e-6,7.68231799535170,9.000000e-03
4,4.553333e-07,MPI_Comm_size(),4.85952941689757e-7,4.90953764193763e-10,0.000988953189305553,4.553333e-07
5,7.433333e-01,MPI_Finalize(),0.488460112539233,130.774528268481,32.7745344861111,7.433333e-01
6,6.153333e-01,MPI_Init(),0.493019444444445,15.0727188036146,30.4268863055556,6.153333e-01
7,5.633333e-02,MPI_Irecv(),0.0607189920652085,-8.96068042327975e-5,-37.4132240107536,5.633333e-02
8,8.950000e-01,MPI_Isend(),3.17498669266321,0.00111110221315045,26.0274814559232,8.950000e-01
9,7.166667e-02,MPI_Reduce(),0.0581820601722947,0.786657959823449,16.4566172194788,7.166667e-02


In [6]:
dict_resultDF[list_resVar[1]]

,real_Inclusive,functionName,predicted_Inclusive,predicted_Inclusive_from_call,predicted_call,real_call
0,4.651960e+03,.TAU_application,4112.13002651492,3.75507819771438,0.000999999999999997,4.651960e+03
1,5.133010e+02,MPI_Allreduce(),439.345968776950,12.1247559208733,69.1144138888889,5.133010e+02
2,3.833333e-02,MPI_Barrier(),0.000869176944444444,0.0185004563418933,0.662485549774719,3.833333e-02
3,9.000000e-03,MPI_Comm_rank(),0.0165754414998621,5.75717034518277e-6,7.68231799535170,9.000000e-03
4,4.553333e-07,MPI_Comm_size(),4.85952941689757e-7,4.90953764193763e-10,0.000988953189305553,4.553333e-07
5,7.433333e-01,MPI_Finalize(),0.501003301920031,130.774528268481,32.7745344861111,7.433333e-01
6,6.153333e-01,MPI_Init(),0.503405555555556,15.0727188036146,30.4268863055556,6.153333e-01
7,8.066667e-02,MPI_Irecv(),0.0635597911803491,-8.96068042327975e-5,-37.4132240107536,8.066667e-02
8,8.950000e-01,MPI_Isend(),0.426386241911113,0.00111110221315045,26.0274814559232,8.950000e-01
9,7.166667e-02,MPI_Reduce(),0.0600932981444429,0.786657959823449,16.4566172194788,7.166667e-02


In [7]:
dict_resultDF[list_resVar[2]]

,real_#Subrs,functionName,predicted_#Subrs,predicted_#Subrs_from_call,predicted_call,real_call
0,1.000000e-03,.TAU_application,0.000999999999999997,9.99999999999994e-7,0.000999999999999997,1.000000e-03
1,0.000000e+00,MPI_Allreduce(),1012.88174466418,0,69.1144138888889,0.000000e+00
2,0.000000e+00,MPI_Barrier(),-0.0124034790238095,0,0.662485549774719,0.000000e+00
3,0.000000e+00,MPI_Comm_rank(),8.33333333333333e-6,0,7.68231799535170,0.000000e+00
4,0.000000e+00,MPI_Comm_size(),0,0,0.000988953189305553,0.000000e+00
5,0.000000e+00,MPI_Finalize(),10.7810178333333,0,32.7745344861111,0.000000e+00
6,0.000000e+00,MPI_Init(),10.8026457222222,0,30.4268863055556,0.000000e+00
7,1.360193e+01,MPI_Irecv(),0.609931260790102,0,-37.4132240107536,1.360193e+01
8,0.000000e+00,MPI_Isend(),27.1089717494639,0,26.0274814559232,0.000000e+00
9,0.000000e+00,MPI_Reduce(),0.0576759273283491,0,16.4566172194788,0.000000e+00
